In [11]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import datetime
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import json
init_notebook_mode(connected=True)
import pandas as pd
from flask import Flask, render_template, request

# 데이터 전처리

In [18]:
df = pd.read_excel('./dataset/syc_df.xlsx')
assign_list = []
for i in range(0,5):
    a = pd.read_excel('./content/TOPIC_XLSX/할당문서_%s.xlsx'%i)
    assign_list.append(list(a.index))
    
key = dict()
for i in range(0,5):
    key[i] = pd.read_excel('./content/TOPIC_XLSX/단어빈도수_%s.xlsx'%i)[:5]

hj_graph_list=[]
for i in range(0,5):
    df1 = df[df.index.isin(assign_list[i])]
    count = df1.groupby(['접수일자']).size().reset_index(name="count")
    hj_graph_list.append(count)
    
df['시간'] =df['접수시각'].apply(lambda x: x[:2]) 
df['월'] = df['datetime'].apply(lambda x:x[:7])
hj_graph = df.groupby(['월']).size().reset_index(name="count")

In [17]:
df.head()

,datetime,title,service,voc_type,voc_date,text,접수일자,요일,접수시각,년도,월,일,년월,년도월일,시간
0,2019-03-21 오전 11:51:51,셔틀 좌석이 고장나있습니다,> >,NaN,NaN,좌측 통로쪽 6번째쯤 좌석이 고정이안되어 등받이가 뒤로 계속 밀립니다원래배차받던 차...,2019-03-21,3,11:51:51,2019,2019-03,21,201903,2019-03-21,11
1,2019-03-21 오전 8:12:02,청운 4관 2층에 사는데 불편한 점이 너무 많습니다.,기타 > 이천 > 기숙사,불편 / 불편정도: 0,2019-03-21,설날 전에 떨어진 화장실 물비누는 다시 채워질 생각을 안 합니다.샤워실에 있는 멀티...,2019-03-21,3,08:12:02,2019,NaN,21,201903,2019-03-21,NaN
2,2019-03-20 오후 11:17:46,편의식 선택,식당 > 이천 > 복지관 > 조식,건의,2019-03-20,편의식 건의드립니다.건강식도 좋지만 우선 배가 안차요.. 우선 배가 든든해야 일을 ...,2019-03-20,2,23:17:46,2019,NaN,20,201903,2019-03-20,NaN
3,2019-03-20 오후 6:05:06,통신사 KT,기타 > 청주 > 기타,불편 / 불편정도: 5,2019-03-20,통신사 KT를 사용중인데 너무 안터져서 확인해보니 SK에서 KT 통신망을 설치 못하...,2019-03-20,2,18:05:06,2019,NaN,20,201903,2019-03-20,NaN
4,2019-03-20 오후 12:58:39,기분이 별로네요,> >,NaN,NaN,아침부터 기분이 안좋네요테이크아웃 바뀌고 좋아져서 식사보다는 샐러드를 이용하거나 테...,2019-03-20,2,24:58:39,2019,NaN,20,201903,2019-03-20,NaN


## 헌준's 그래프 코딩

In [2]:

def make_month_num(df):
    ### return data which has month_number
    date_data = pd.to_datetime(df['접수일자'])
    month_num = date_data.dt.strftime('%Y-%m')
    df['month_num'] = month_num
    df_sum_month = df.groupby(['month_num']).sum()
    df_sum_month['month_num']= df_sum_month.index
    df_sum_month.index = list(range(len(df_sum_month)))
    return df_sum_month


def make_week_num(df):
    ### return data which has week_number
    date_data = pd.to_datetime(df['접수일자'])
    week_num = date_data.dt.strftime('%Y-W%U')
    df['week_num'] = week_num
    df_sum_week = df.groupby(['week_num']).sum()
    df_sum_week['week_num']= df_sum_week.index
    df_sum_week.index = list(range(len(df_sum_week)))
    week_num_new = []
    for i in df_sum_week['week_num']:
        week_num_new.append(datetime.datetime.strptime(i + '-1', "%Y-W%W-%w"))
    df_sum_week['week_num_new'] = week_num_new
    return df_sum_week

    week_scatter_plot  = go.Scatter(x=week_df['week_num_new'], y=week_df['count'])
    layout = go.Layout(
        font={
            "family": "뫼비우스 Regular",
            "size": 15
        },

        title={
            "text": '시간 흐름에 따른 관심도 변화, 주별',
            "font": {
                "size": 24,

            }
        },
        xaxis=dict(
            title='Date',
            titlefont=dict(
                family='뫼비우스 Regular',
                size=18,
                color='#7f7f7f')
        ),
        yaxis=dict(
            title='Count',
            titlefont=dict(
                family='뫼비우스 Regular',
                size=18,
                color='#7f7f7f')
        ),
        hoverlabel={
            "font": {
                "family": "뫼비우스 Regular"}
        }
    )

    fig2 = go.Figure(data=week_scatter_plot, layout=layout)
    fig1 = json.dumps(fig1, cls=plotly.utils.PlotlyJSONEncoder)
    fig2 = json.dumps(fig2, cls=plotly.utils.PlotlyJSONEncoder)
    
    ## Monthly Trend plot
    return [fig1,fig2]



##########################################################################################
############### main function
def hj_trend_chart(df):
    #### Description
    ## input : file name list
    ##         file must contain two columns whose name
    ##          date     : 일자
    ##          count : 일자별 count
    ## return : Null
    ## print two trend charts : Monthly, Weekly
    ## for 5 topic categories.
    #### Example
    ## file_name = ['filename1.xlsx','filename2.xlsx']
    ## hj_trend_chart(file_name)

    #### data prepocessing : data -> week_df, month_df
    # df = []
    # for i in file_names:
    #    df.append(pd.read_excel(i))

    month_df = []
    for i in df:
        month_df.append(make_month_num(i))

    week_df = []
    for i in df:
        week_df.append(make_week_num(i))


    ### Monthly trend plot
    month_scatter_plot = []
    for i in range(len(month_df)):
        month_scatter_plot.append(go.Scatter(
            x=list(month_df[i]['month_num']),
            y=list(month_df[i]['count']),
            name='Cate' + str(i),
            line=dict(width=1.5)))

    layout = go.Layout(
        font={
            "family": "뫼비우스 Regular",
            "size": 15
        },

        title={
         "text": '시간 흐름에 따른 관심도 변화, 월별',
            "font": {
                "size": 24,

            }
        }
        ,
        xaxis=dict(
            title='Date',
            titlefont=dict(
             family='뫼비우스 Regular',
             size=18,
             color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Count',
            titlefont=dict(
                family='뫼비우스 Regular',
                size=18,
                color='#7f7f7f'
            )
        ),
        hoverlabel={
            "font": {
                "family": "뫼비우스 Regular",

            }
        }
    )

    fig1 = go.Figure(data=month_scatter_plot, layout=layout)
    # ploting monthly trend chart

    ## Weekly Trend plot
    week_scatter_plot = []
    for i in range(len(week_df)):
        week_scatter_plot.append(go.Scatter(
            x=list(week_df[i]['week_num_new']),
            y=list(week_df[i]['count']),
            name='Cate' + str(i),
            line=dict(width=1.5)))


        layout = go.Layout(
            font={
                "family": "뫼비우스 Regular",
                "size": 15
            },

            title={
                "text": '시간 흐름에 따른 관심도 변화, 주별',
                "font": {
                    "size": 24,

                }
            },
            xaxis=dict(
                title='Date',
                titlefont=dict(
                    family='뫼비우스 Regular',
                    size=18,
                    color='#7f7f7f')
            ),
            yaxis=dict(
                title='Count',
                titlefont=dict(
                    family='뫼비우스 Regular',
                    size=18,
                    color='#7f7f7f')
            ),
            hoverlabel={
                "font": {
                    "family": "뫼비우스 Regular"}
            }
        )

    fig2 = go.Figure(data=week_scatter_plot, layout=layout)
    fig1 = json.dumps(fig1, cls=plotly.utils.PlotlyJSONEncoder)
    fig2 = json.dumps(fig2, cls=plotly.utils.PlotlyJSONEncoder)
    ## Monthly Trend plot
    return [fig1,fig2]

In [3]:
def create_plot(df):

    layout = go.Layout(
        font={
            "family": "뫼비우스 Regular",
            "size": 15
        },

        title={
         "text": '시간 흐름에 따른 관심도 변화, 월별',
            "font": {
                "size": 24,

            }
        }
        ,
        xaxis=dict(
            title='Date',
            titlefont=dict(
             family='뫼비우스 Regular',
             size=18,
             color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Count',
            titlefont=dict(
                family='뫼비우스 Regular',
                size=18,
                color='#7f7f7f'
            )
        ),
        hoverlabel={
            "font": {
                "family": "뫼비우스 Regular",

            }
        }
    )
    
#     iplot([{"x": df['monthonly'], "y": df['counts']}])
    data = [go.Scatter(x=df['월'] ,y=df['count'])]
    fig = go.Figure(data=data, layout=layout)
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
#     df = pd.read_csv('./stat_perday.csv')
#     data = [go.Scatter(x=df['dateonly'] ,y=df['counts'])]
#     graphJSON2 = json.dumps(data, cls=plotly.utils.PlotlyJSONEncoder)

    return graphJSON

In [4]:
def plot_allgraph(idx):
    df1 = df[df.index.isin(assign_list[idx])]
#     df1['시간'] =df1['접수시각'].apply(lambda x: x[:2]) 
#     df1['월'] = df1['datetime'].apply(lambda x:x[:7])
    stat_time = df1.groupby(['시간']).size().reset_index(name='counts')
    stat_day = df1.groupby(['요일']).size().reset_index(name='counts')
    stat_month = df1.groupby(['월']).size().reset_index(name='counts')
    
    tra = go.Bar(x=["월","화","수","목","금","토","일"],y=stat_day['counts'],marker={'color':'rgba(26, 100, 219, 0.7)'})
    lay = go.Layout(font={"family":"뫼비우스 Regular","size":15},title="Topic 0의 요일 별 통계",xaxis={"title":"요일"})
    fig = go.Figure(data=[tra], layout=lay)
    graph_day = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    
    tra = go.Bar(x=stat_time['시간'],y=stat_time['counts'],marker={'color':'rgba(26, 100, 219, 0.7)'})
    lay = go.Layout(font={"family":"뫼비우스 Regular","size":15},title="Topic 0의 시간 별 통계",xaxis={"title":"시간"})
    fig = go.Figure(data=[tra], layout=lay)
    graph_time = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)


    tra = go.Scatter(x=stat_month['월'],y=stat_month['counts'],marker={'color':'rgba(245,128,37, 0.7)'})
    lay = go.Layout(font={"family":"뫼비우스 Regular","size":15},title="Topic 0의 월 별 통계",xaxis={"title":"날짜"})
    fig = go.Figure(data=[tra], layout=lay)
    graph_month = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    
    return graph_day, graph_time, graph_month

In [21]:
##### from flask import Flask, render_template, request


app = Flask(__name__)

@app.route("/all_graph")
def allgraph():
    graph_list = []
    key_list = []
    do_list = []
    for i in range(0,5):
        gr1,gr2, gr3 = plot_allgraph(i)
        graph_list.append([gr1,gr2,gr3])
        key_list.append(key[i].values)
        do_list.append(donut_plot)

    return render_template('tot_info.html',plot=graph_list, key=key_list, donut= do_list)

@app.route("/graph")
def graph():

#     gr1,gr2 = create_plot()
    hj_graph3 = create_plot(hj_graph)
    hj_graph1, hj_graph2 = hj_trend_chart(hj_graph_list)
    return render_template('graph.html',plot=hj_graph1,plot2=hj_graph2,plot3=hj_graph3)

@app.route("/")
def index():
    
    return render_template("index.html")

@app.route("/index")
def index2():
    
    return render_template("index.html")

@app.route("/keyword")
def keyword():
    
    return render_template("keyword.html")

@app.route("/hi",methods=['GET','POST'])
def hi():

    return render_template("submit.html")

@app.route("/select")
def select():

    return render_template("select.html")

@app.route("/dash")
def dash():

    return render_template("dashboard.html")


@app.route("/test",methods=['GET','POST'])
def test():
    if request.method == 'POST':
        result = request.form
        start = result['from']
        end = result['to']
        num = result['topic_num']
        new_result=dict()
        new_result['from'] = str(start)+"nnnn"
        new_result['end'] = str(end)+"kkkk"
        new_result['num'] = num*1000
    elif request.method=="GET":
        
        new_result=dict()
        start = request.args.get('from')
        end = request.args.get('to')
        num = request.args.get('topic_num')
        
        
        new_result['from'] = str(start)+"nnnn"
        new_result['end'] = str(end)+"kkkk"
        new_result['num'] = num*1000
    
    return render_template("test.html",result = new_result)


if __name__ == "__main__":
    app.run(host='59.29.224.81',threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://59.29.224.81:5000/ (Press CTRL+C to quit)
59.29.224.81 - - [27/Mar/2019 17:39:21] "GET /all_graph HTTP/1.1" 200 -
59.29.224.81 - - [27/Mar/2019 17:39:21] "GET /static/img/wordcloud.png HTTP/1.1" 200 -
59.29.224.81 - - [27/Mar/2019 17:40:18] "GET /static/img/%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C_0.png HTTP/1.1" 200 -
59.29.224.81 - - [27/Mar/2019 17:40:28] "GET /static/img/%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C_4.png HTTP/1.1" 200 -


In [5]:
d1 = pd.read_csv('./crawling.csv')

In [6]:
d2 = d1[d1['voc_type'].isin(['칭찬','불편 - 기타'])]

In [7]:
d3 = d2.groupby(['voc_type']).size().reset_index(name='counts')

In [8]:
labels = list(d3['voc_type'])
values = list(d3['counts'])

In [20]:
tra = go.Pie(labels=labels[:5], values=values[:5], hole=.7, name="Hi")
lay = go.Layout(font={"family":"뫼비우스 Regular"},annotations=[{"text":"버스","x":0.5,"y":0.5,"showarrow":False,"font":{"family":"뫼비우스 Regular","size":50,"color":"black"}}])
fig = go.Figure(data=[tra], layout=lay)
donut_plot =  json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)